# Preprocessing intracranial EEG using MNE-python


*WIRED 2024*  
[Maansi Desai, PhD](https://maansidesai.github.io/)  
Postdoctoral Researcher in the [Hamilton Lab](https://slhs.utexas.edu/research/hamilton-lab)
<br>
Department of Speech, Language, and Hearing Sciences 
<br>
The University of Texas at Austin 

## Part 1: Loading, Plotting, and Referencing
This notebook will show you how to preprocess intracranial EEG using MNE-python. This uses an iEEG dataset on sentence listening which can be accessed [here](https://openneuro.org/datasets/ds003688/versions/1.0.7/metadata). This notebook covers the basics of how to look at iEEG data, interpret effects of referencing, and inspect data quality. In part 2, we will cover high gamma extraction and referencing. The method of high gamma extraction is identical to that used in [Hamilton et al. 2018](https://doi.org/10.1016/j.cub.2018.04.033) and [Hamilton et al. 2021](https://doi.org/10.1016/j.cell.2021.07.019).

## Python libraries used in this tutorial

* matplotlib
* mne_bids
* [MNE-python](https://mne.tools/stable/install/index.html)
* numpy
* pandas
* pybids

## What you will do in this tutorial

* Load an iEEG dataset in MNE-python
* Plot the power spectrum of the data to check for bad channels and compare channel types
* Re-reference the data according to different reference schemes
* Based on the functions and tools you will learn, there will be some portions throughout the tutorial where we will ask you to test out your knowledge in areas of the notebook which say `Try on your own` 

### part 2! ([`02_ieeg_preprocessing_MNE_epochs.ipynb`](02_ieeg_preprocessing_MNE_epochs.ipynb))
* Compute the high gamma analytic amplitude of the signal
* Plot evoked data

### part 3! ([`03_plot_electrodes.ipynb`](03_plot_electrodes.ipynb)) -- visualization tools [optional]
* load electrode coordinates stored in imaging from BIDS data and plot location on the brain for visualization purposes
* This notebook will mostly be a resource to show you how to plot electrodes on the brain and will also allow you to plot evoked responses on the brain as well. 
* We will not be going over this notebook during this workshop, however we encourage you to take a look at this code on your own time.

# A high-level summary for preprocessing intracranial data

### Frequency Bands

EEG records neural responses best in the range of 1-25Hz. However, ECoG can record from a much higher range of frequency, up into gamma and high gamma (70-150Hz) bands. This is especially exciting for speech researchers, because high-gamma band activity has been correlated with a lot of speech stimuli!

However, to extract high gamma response we need to do something called a Hilbert transform. We'll talk more about that in `02_ieeg_preprocessing_MNE_epochs` notebook. 

### Artifacts

ECoG is invasive, that is, it's placed directly on the surface of the brain (or in the brain, in the case of depth electrodes). Some of the artifacts we may encounter are related to the patient's epilepsy or other movement-related noise. We'll go over examples of what these artifacts look like. 

### Referencing

There are different methods that people use for rereferencing intracranial data. For sEEG/ECoG instead we use a common average reference (CAR), which means we are referencing based on the average activity at all electrodes. But we will go over a couple of rereferencing methods and will plot the neural data to give you an idea of how the data look.

In [1]:
#%matplotlib notebook
#%matplotlib tk

import mne
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from mne_bids import read_raw_bids, print_dir_tree
from mne_bids.path import get_bids_path_from_fname
from bids import BIDSLayout
from ecog_preproc_utils import transformData
import bids
import re  # regex for comparing channel names

print(mne.__version__)

1.6.1


## Download BIDS iEEG dataset

Here we will download an example iEEG dataset from some [pediatric intracranial data collected from the Hamilton Lab](https://openneuro.org/datasets/ds004993/versions/1.0.2). 

For this tutorial we will use data from: 
 - `sub-W1`, `iemu` data only, for detecting seizure activity. 
 - `sub-W2`, `iemu` data only, for epoching data to the onset of the speech stimuli (from the [TIMIT speech corpus](https://catalog.ldc.upenn.edu/LDC93S1)) and plotting the evoked responses in the `02_ieeg_preprocessing_MNE_epochs` notebook. 
 - `sub-W3`, `iemu` data only, for detecting artifacts from a vagus nerve stimulator (VNS)

In [2]:
# This is the example participant's data that we will load for the tutorial,
# but there are more options.
subj = 'W2'
sess = 'iemu'
task = 'timit5'
acq = 'B8'
run = '01'

# Change the data directory below to where your data are located. 
parent_dir = 'data/ds004993'  # This is on your local machine

In [3]:
ieeg_dir = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
channel_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_channels.tsv'
raw_path = f'{ieeg_dir}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_ieeg.edf'


bids_path = get_bids_path_from_fname(raw_path)
base_name = os.path.basename(raw_path).split('.')[0]

print(bids_path)

data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_ieeg.edf


**Remark:** Note that bids_path is an instance of [BIDSPath](https://mne.tools/mne-bids/dev/generated/mne_bids.BIDSPath.html) object in mne_bids:

In [4]:
bids_path

BIDSPath(
root: data/ds004993
datatype: ieeg
basename: sub-W2_ses-iemu_task-timit5_acq-B8_run-01_ieeg.edf)

In [5]:
bids_path.subject, bids_path.session

('W2', 'iemu')

## BIDS layout

We can use `pybids` to show a little bit about the files in this BIDS dataset. We won't get as much into this, but if you'd like to try this tutorial on your own you may wish to delve into this more.

In [6]:
layout = BIDSLayout(parent_dir)

In [7]:
layout.get_tasks()

['movietrailers', 'timit5', 'timit4']

In [8]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 10 files are:")
all_files[:10]

There are 30 files in the layout.

The first 10 files are:


[<BIDSFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/CHANGES'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/dataset_description.json'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/participants.json'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/participants.tsv'>,
 <BIDSFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/README'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W1/ses-iemu/ieeg/sub-W1_ses-iemu_task-movietrailers_acq-B3_run-01_channels.tsv'>,
 <BIDSJSONFile filename='/Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W1/ses-iemu/ieeg/sub-W1_ses-iemu_task-movietrailers_acq-B3_run-01_coordsystem.json'>,
 <BIDSDataFile filename='/Users/maansidesai/Desktop/git/WIRED-2

In [9]:
print_dir_tree(parent_dir, max_depth=3)

|ds004993/
|--- .DS_Store
|--- .bidsignore
|--- CHANGES
|--- README
|--- dataset_description.json
|--- participants.json
|--- participants.tsv
|--- freesurfer/
|------ .DS_Store
|------ sub-W1/
|--------- W1_lateral_lh.png
|--------- W1_lateral_rh.png
|--------- W1_ventral.png
|--------- bem/
|--------- surf/
|------ sub-W2/
|--------- W2_lateral_rh.png
|--------- W2_ventral_rh.png
|--------- bem/
|--------- surf/
|------ sub-W3/
|--------- .DS_Store
|--------- W3_frontal.png
|--------- W3_lateral_lh.png
|--------- W3_lateral_rh.png
|--------- bem/
|--------- surf/
|--- sub-W1/
|------ ses-iemu/
|--------- sub-W1_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|--------- anat/
|--- sub-W2/
|------ ses-iemu/
|--------- sub-W2_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|--------- anat/
|--- sub-W3/
|------ ses-iemu/
|--------- sub-W3_ses-iemu_scans.tsv
|--------- ieeg/
|------ ses-mri/
|--------- anat/


## Let's load some iEEG data!

First, we will choose the relevant subject, session, task, acquisition, and run. Note that if you wish to change these variables, you may need to download the data yourself.

To show the capabilities of BIDS and contrast to when we don't use BIDS, we'll load the data in two ways. The data structure using BIDS will be called `raw`.

In [10]:
# Read data and extract parameters from BIDS files
raw = read_raw_bids(bids_path, verbose=True)


Extracting EDF parameters from /Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_events.tsv.
Reading channel info from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_channels.tsv.
Reading electrode coords from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info


/var/folders/03/nrm13jr11b734k78bp62stg80000gq/T/ipykernel_55590/4023317972.py:2: RuntimeWarning: Omitted 102 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path, verbose=True)


In [11]:
# Let's load the data into memory and print some information about it. The 
# info structure contains a lot of helpful metadata about number of channels,
# sampling rate, data types, etc. It can also contain information about the
# participant and date of acquisition, however, this dataset has been anonymized.
raw.load_data()
raw.info

Reading 0 ... 182783  =      0.000 ...   356.998 secs...


Measurement date,"January 01, 2024 00:00:00 GMT"
Experimenter,mne_anonymize
Participant,sub-W2
Digitized points,106 points
Good channels,106 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,512.00 Hz
Highpass,0.00 Hz
Lowpass,256.00 Hz


In [12]:
raw.info['ch_names']

['RAST1',
 'RAST2',
 'RAST3',
 'RAST4',
 'RAST5',
 'RAST6',
 'RAST7',
 'RAST8',
 'RAST9',
 'RAST10',
 'RAST11',
 'RAST12',
 'RMST1',
 'RMST2',
 'RMST3',
 'RMST4',
 'RMST5',
 'RMST6',
 'RMST7',
 'RMST8',
 'RMST9',
 'RMST10',
 'RMST11',
 'RMST12',
 'RMST13',
 'RMST14',
 'RPST1',
 'RPST2',
 'RPST3',
 'RPST4',
 'RPST5',
 'RPST6',
 'RPST7',
 'RPST8',
 'RPST9',
 'RPST10',
 'RPST11',
 'RPST12',
 'RPPST1',
 'RPPST2',
 'RPPST3',
 'RPPST4',
 'RPPST5',
 'RPPST6',
 'RPPST7',
 'RPPST8',
 'RPPST9',
 'RPPST10',
 'RPPST11',
 'RPPST12',
 'RPPST13',
 'RPPST14',
 'RPPST15',
 'RPPST16',
 'RAIF1',
 'RAIF2',
 'RAIF3',
 'RAIF4',
 'RAIF5',
 'RAIF6',
 'RAIF7',
 'RAIF8',
 'RPI1',
 'RPI2',
 'RPI3',
 'RPI4',
 'RPI5',
 'RPI6',
 'RPI7',
 'RPI8',
 'RPI9',
 'RPI10',
 'RPI11',
 'RPI12',
 'RPI13',
 'RPI14',
 'RPI15',
 'RPI16',
 'ROF1',
 'ROF2',
 'ROF3',
 'ROF4',
 'ROF5',
 'ROF6',
 'ROF7',
 'ROF8',
 'ROF9',
 'ROF10',
 'ROF11',
 'ROF12',
 'ROF13',
 'ROF14',
 'ROF15',
 'ROF16',
 'RAC1',
 'RAC2',
 'RAC3',
 'RAC4',
 'RAC5',

## Plot the raw data

Let's first inspect the raw data to see how it looks, what type of information we have, and whether we can immediately see any bad channels or bad time segments that should be rejected. Typically this portion should be done interactively so you can scroll through the data using the arrow keys. 

Compare and contrast the data loaded using mne-bids (`raw`) versus the data loaded without this information.
To get a raw object using mne-bids you can use the `read_raw_bids` function. See 2 cells above.

In [13]:
%matplotlib qt
# mne.viz.set_browser_backend('matplotlib')
raw.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True);

Using qt as 2D backend.
Using pyopengl with version 3.1.6


qt.qpa.openglcontext: Could not find virtual screen for QCocoaScreen(0x7f99d8c64210, "", QRect(-1920,0 1920x1080), dpr=1, displayId=4128833, <NSScreen: 0x7f99d8c61080>) with displayId 4128833
qt.qpa.openglcontext: Could not find virtual screen for QCocoaScreen(0x7f99d8c64210, "", QRect(-1920,0 1920x1080), dpr=1, displayId=4128833, <NSScreen: 0x7f99d8c61080>) with displayId 4128833
qt.qpa.openglcontext: Could not find virtual screen for QCocoaScreen(0x7f99d8c64210, "", QRect(-1920,0 1920x1080), dpr=1, displayId=4128833, <NSScreen: 0x7f99d8c61080>) with displayId 4128833


Channels marked as bad:
none
Channels marked as bad:
none
Channels marked as bad:
none


**NOTE:** From now on we will be using the mne-bids version.

## Plot the power spectrum

Now we will plot the power spectrum of the signal to give us an idea of the signals we're getting. Bad channels (or channels that are not EEG/ECoG) will often have a very different power spectrum than the good channels. These will show up as highly outside the range of the other channels (either flat, or much higher/lower power).

In [14]:
raw = read_raw_bids(bids_path, verbose=True)  # let's reload so we are sure to all use the BIDS version.
raw.load_data()

Extracting EDF parameters from /Users/maansidesai/Desktop/git/WIRED-2024-Paris/notebooks/data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_events.tsv.
Reading channel info from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_channels.tsv.
Reading electrode coords from data/ds004993/sub-W2/ses-iemu/ieeg/sub-W2_ses-iemu_task-timit5_acq-B8_run-01_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading 0 ... 182783  =      0.000 ...   356.998 secs...


/var/folders/03/nrm13jr11b734k78bp62stg80000gq/T/ipykernel_55590/3090690490.py:1: RuntimeWarning: Omitted 102 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path, verbose=True)  # let's reload so we are sure to all use the BIDS version.


Measurement date,"January 01, 2024 00:00:00 GMT"
Experimenter,mne_anonymize
Participant,sub-W2
Digitized points,106 points
Good channels,106 sEEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,512.00 Hz
Highpass,0.00 Hz
Lowpass,256.00 Hz


In [15]:
raw.compute_psd().plot(picks='data', exclude=[]);

Effective window size : 4.000 (s)


In [16]:
# If we want to see other options we have for computing the power spectrum, 
# we can consult the help function
raw.compute_psd?

Signature:
raw.compute_psd(
    method='welch',
    fmin=0,
    fmax=inf,
    tmin=None,
    tmax=None,
    picks=None,
    exclude=(),
    proj=False,
    remove_dc=True,
    reject_by_annotation=True,
    *,
    n_jobs=1,
    verbose=None,
    **method_kw,
)
Docstring:
Perform spectral analysis on sensor data.

Parameters
----------

method : ``'welch'`` | ``'multitaper'``
    Spectral estimation method. ``'welch'`` uses Welch's
    method :footcite:p:`Welch1967`, ``'multitaper'`` uses DPSS
    tapers :footcite:p:`Slepian1978`.
    Default is ``'welch'``.
fmin, fmax : float
    The lower- and upper-bound on frequencies of interest. Default is ``fmin=0, fmax=np.inf`` (spans all frequencies present in the data).
tmin, tmax : float | None
    First and last times to include, in seconds. ``None`` uses the first or
    last time present in the data. Default is ``tmin=None, tmax=None`` (all
    times).
picks : str | array-like | slice | None
    Channels to include. Slices and lists of int

## Referencing

Referencing or re-referencing your data should be done with some knowledge of your recording setup and what you wish to measure. You can read more about referencing [here (for EEG)](https://pressrelease.brainproducts.com/referencing/#:~:text=The%20reference%20influences%20the%20amplitude,affected%20by%20similar%20electrical%20activity.). Typically, experimenters will choose one of the following references:

1. Based on a single electrode in white matter (or relatively "quiet" electrode far away from your signals of interest).
2. Based on the average of all electrodes or a block of electrodes (CAR or Common Average Reference). Note that the CAR is *not* a good idea if all of your electrodes are within a single functional area, as you will likely subtract out more signal than noise. 
3. Bipolar referencing, in which pairs of adjacent electrodes are subtracted to calculate more local signals. This is a bit more complicated but allows you to work with data in a single region without the drawbacks of the CAR.

In [17]:
# Example of single reference channel - this subtracts the signal from this channel
# from all other channels (including itself), so the reference will appear as a flat
# line after this step
raw_ref = raw.copy()
raw_ref.set_eeg_reference(ref_channels = ['RAC12'], ) #Here we are just picking a channel that looks relatively noise-free 
raw_ref.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=False);

sEEG channel type selected for re-referencing
Applying a custom ('sEEG',) reference.
Using pyopengl with version 3.1.6


In [18]:
# Example of common average reference. The common average reference subtracts the average
# signal across all *good* channels from every single channel. This is typically a good
# choice for removing noise across all channels (for example, sometimes EMG from chewing,
# some movement artifacts, electrical noise).
raw_ref_car = raw.copy()
raw_ref_car.set_eeg_reference(ref_channels = 'average')
raw_ref_car.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=False)

sEEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('sEEG',) reference.
Using pyopengl with version 3.1.6


## Bipolar reference

Bipolar referencing is a bit trickier and is not fully implemented here. You need to use knowledge of the physical locations of the electrodes to properly create the bipolar montage. For example, in the image below, we would need to use the knowledge of how the electrodes are placed in order to create the appropriate pairs for the anode and cathode.

![sub-06 electrode locations](sub-06_ses-iemu_acq-render_photo_ecog_left.jpeg)

In [19]:
# Example of bipolar reference. This would normally be done with some manual
# intervention of the specific channel pairs. 
raw_ieeg = raw.copy()
# raw_ieeg.pick_types(eeg=True)
ch_pairs = list(zip(raw_ieeg.info['ch_names'][:-1],
                    raw_ieeg.info['ch_names'][1:]))

# Make a list of channels for the anode and the cathode
anode = []
cathode = []
# Only subtract channels with the same device name (these will be
# close in space). This is still not ideal as we are probably
# subtracting electrodes that are far away from one another in 
# space (for example, electrodes 8 and 9 on the grid picture
# above should not be used for a bipolar reference)
for pair in ch_pairs:
    # get rid of the numbers in the ch_name
    ch1_dev = re.sub(r'\d+', '', pair[0]) 
    ch2_dev = re.sub(r'\d+', '', pair[1]) 
    # if these are part of the same device, consider them for 
    # anode and cathode selection
    if ch1_dev == ch2_dev:
        anode.append(pair[0])
        cathode.append(pair[1])

# Apply the bipolar reference
raw_ref_bip = mne.set_bipolar_reference(raw, anode=anode, cathode=cathode)
raw_ref_bip.plot(scalings='auto', color=dict(eeg='b', ecog='b'), n_channels=64, block=True);

sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=98, n_times=182784
    Range : 0 ... 182783 =      0.000 ...   356.998 secs
Ready.
Added the following bipolar channels:
RAST1-RAST2, RAST2-RAST3, RAST3-RAST4, RAST4-RAST5, RAST5-RAST6, RAST6-RAST7, RAST7-RAST8, RAST8-RAST9, RAST9-RAST10, RAST10-RAST11, RAST11-RAST12, RMST1-RMST2, RMST2-RMST3, RMST3-RMST4, RMST4-RMST5, RMST5-RMST6, RMST6-RMST7, RMST7-RMST8, RMST8-RMST9, RMST9-RMST10, RMST10-RMST11, RMST11-RMST12, RMST12-RMST13, RMST13-RMST14, RPST1-RPST2, RPST2-RPST3, RPST3-RPST4, RPST4-RPST5, RPST5-RPST6, RPST6-RPST7, RPST7-RPST8, RPST8-RPST9, RPST9-RPST10, RPST10-RPST11, RPST11-RPST12, RPPST1-RPPST2, RPPST2-RPPST3, RPPST3-RPPST4, RPPST4-RPPST5, RPPST5-RPPST6, RPPST6-RPPST7, RPPST7-RPPST8, RPPST8-RPPST9, RPPST9-RPPST10, RPPST10-RPPST11, RPPST11-RPPST12, RPPST12-RPPST13, RPPST13-RPPST14, RPPST14-RPPST15, RPPST15-RPPST16, RAIF1-RAIF2, RAIF2-RAIF3, RAIF3-RAIF4, RAIF4-RAIF5, RAIF5-RAIF6, RAIF6-RA

# Now let's look at other types of artifacts that you may encounter while preprocessing intracranial data

In [20]:
# We're going to start by loading another patient from our BIDS data

subj = 'W1'
sess = 'iemu'
task = 'movietrailers'
acq = 'B3'
run = '01'

# Change the data directory below to where your data are located. 
parent_dir = 'data/ds004993'  # This is on your local machine

ieeg_dir_epi = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
channel_path_epi = f'{ieeg_dir_epi}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_channels.tsv'
raw_path_epi = f'{ieeg_dir_epi}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_ieeg.edf'

bids_path_epi = get_bids_path_from_fname(raw_path_epi)
base_name_epi = os.path.basename(raw_path_epi).split('.')[0]

print(bids_path_epi)
print('I ran "black" formatting on the .py files')

data/ds004993/sub-W1/ses-iemu/ieeg/sub-W1_ses-iemu_task-movietrailers_acq-B3_run-01_ieeg.edf
I ran "black" formatting on the .py files


## This specific block of data will show what seizure activity looks like:

#### Try on your own: Take a moment and try to plot the raw data on your own. 

Hint: Use `bids_path_epi`

In [24]:
# Plot the data, reject bad segments. Look for times where there
# are spike wave discharges (epileptiform artifacts) or large
# movement artifacts. Be selective, look out for blocks with a 
# ton of seizure activity

### YOUR CODE HERE ### (bids_path_epi)


## Epileptiform activity

This is an example in one patient who is having a seizuring during one of our tasks. From this picture and the data you were able to scroll through above, you can see that there is a cyclical pattern of noise that is across all channels. In these cases,  you would not reject these channels only. Instead, you unfortunately would have to eliminate this particular recording block from your analysis. 

![epileptic artifact](epileptic_artifact.jpeg)


## Vagus Nerve stimulator (VNS) artifact 

When working with intractable epilepsy patients, one of the factors that may create artifacts can arise from a [VNS device](https://www.epilepsy.com/treatment/devices/vagus-nerve-stimulation-therapy). In short, a VNS is a type of neuromodulation that delivers stimulation to the brain to specific parts of the brain that may cause epilepsy. Not all patients have a VNS, but you are working with a patient who does, we have found that there can be 20 Hz noise that appears when collecting data in the EMU. Let's take a look at an example.

In [23]:
#We are going to load another patient's data that is stored in our BIDS dataset.  

subj = 'W3'
sess = 'iemu'
task = 'timit4'
acq = 'B8'
run = '01'
parent_dir = 'data/ds004993'  # This is on your local machine

print(parent_dir)
ieeg_dir_vns = f'{parent_dir}/sub-{subj}/ses-{sess}/ieeg/'
channel_path_vns = f'{ieeg_dir_vns}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_channels.tsv'
raw_path_vns = f'{ieeg_dir_vns}/sub-{subj}_ses-{sess}_task-{task}_acq-{acq}_run-{run}_ieeg.edf'


bids_path_vns = get_bids_path_from_fname(raw_path_vns)
base_name_vns = os.path.basename(raw_path_vns).split('.')[0]

print(raw_path_vns)
print(base_name_vns)

data/ds004993
data/ds004993/sub-W3/ses-iemu/ieeg//sub-W3_ses-iemu_task-timit4_acq-B8_run-01_ieeg.edf
sub-W3_ses-iemu_task-timit4_acq-B8_run-01_ieeg


#### Try on your own:  Once again, take a moment and try to plot the raw data on your own. 

Hint: Use `bids_path_vns`

In [27]:
# Plot the data, to view VNS artifact

### YOUR CODE HERE ### 


## VNS electrical activity

This is an example in one patient that has electrical artifact from their VNS system

![VNS artifact](vns_artifact.jpeg)


In [26]:
raw_vns.compute_psd().plot(picks='data', exclude=[]);

Effective window size : 1.000 (s)


## VNS power spectrum

When we plot the power spectrum, we see consistent 20 Hz noise which is from the VNS neurostimulator. 

![VNS power spectrum](vns_power_spectrum.jpeg)


## If you see electrical artifact from a VNS, such as this example, you don't need to reject every single time point when you plot the neural data for artifact rejection. Instead, performing referencing by using a common average reference (CAR) will help to eliminate this noise.

#### Try on your own: Take a moment and try to plot the raw data on your own. 

Try to perform car on the `raw_vns` data. Then plot the data to see what happens

In [ ]:
### YOUR CODE ###   #perform a common average reference


### YOUR CODE ###   # Plot the raw data now

In [ ]:
### YOUR CODE ###   # Plot the raw data now


## Want to plot some evoked data? 

Go to Part 2! [02_ieeg_preprocessing_MNE_epochs.ipynb](02_ieeg_preprocessing_MNE_epochs.ipynb)